In [ ]:
# This notebook is configured to run on the VM

## Import our data

In [ ]:
#Import libraries
from io import StringIO
import requests
import json
import pandas as pd

In [ ]:
#Note all dataframes should be named in respect to the below dataframe names
# if inserted Pandas DataFrame, skip or delete this cell
df_data_1 = pd.read_csv('../data/opioids.csv')
df_data_1.head()

In [ ]:
# if inserted Pandas DataFrame, skip or delete this cell
df_data_2 = pd.read_csv('../data/overdoses.csv')
df_data_2.head()

In [ ]:
# if inserted Pandas DataFrame, skip or delete this cell
df_data_3 = pd.read_csv('../data/prescriber-info.csv')
df_data_3.head()

## Exploration and Initial Preprocessing

In [ ]:
# Let's start out by removing the ',' from our numbers in the Deaths and Population columns so that we can use them as integers
df_data_2['Deaths'] = df_data_2['Deaths'].str.replace(',', '')
df_data_2['Deaths'] = df_data_2['Deaths'].astype(int)

In [ ]:
df_data_2['Population'] = df_data_2['Population'].str.replace(',', '')
df_data_2['Population'] = df_data_2['Population'].astype(int)

In [ ]:
#Adding an additional column where we see the deaths per capita per each state
df_data_2['Deaths/Population'] = (df_data_2['Deaths']/df_data_2['Population'])

In [ ]:
#Let's check to see that it worked!
df_data_2.head()

In [ ]:
#Use pixiedust to visualize our initial exploration.
import pixiedust

In [ ]:
#How many opioid deaths by U.S. state?
display(df_data_2)

In [ ]:
#It definitely looks like California has a great deal more deaths than any other state. 
#Let's remember, however, California is a huge state with a matching population. Because of this we need to take a look at the values of deaths per capita.

In [ ]:
#What about deaths (% of population) by U.S. State?
display(df_data_2)

In [ ]:
#We can see that West Virginia, New Mexico, New Hampshire, Ohio, Kentucky and Delaware stand out. 

In [ ]:
#Let's check this out with a map. (I used google maps. To do this, create an API and enable JavaScript and GeoCoding. Then use your API key under 'Options'.)
display(df_data_2)

In [ ]:
#Let's move onto exploring our other dataset.
df_data_3.head()

In [ ]:
#We seem to have a great deal of prescriptions as well as physicians' gender, state, speciality, whether they are an opioid prescriber or not and unique ID.
df_data_3.count()

In [ ]:
#Let's take a look at the states. Why are there more than 50 states?
df_data_3.State.unique()

In [ ]:
#Compare to df_data_2.
df_data_2.Abbrev.unique()

In [ ]:
#Clean up states and make the dataset state list equal.
#I checked the list of US state abbreviations and did not recognize PR, AE, ZZ, GU, AA or VI. After checking I learned that PR is Puerto Rico, GU is Guam and VI is Virgin Islands.
#Though I identified 3 of the 6 unknowns, I'll remove all of them as dataset 2 does not have data regarding PR, GU or VI.
df_data_3 = df_data_3[df_data_3.State != 'AE']
df_data_3 = df_data_3[df_data_3.State != 'ZZ']
df_data_3 = df_data_3[df_data_3.State != 'AA']
df_data_3 = df_data_3[df_data_3.State != 'PR']
df_data_3 = df_data_3[df_data_3.State != 'GU']
df_data_3 = df_data_3[df_data_3.State != 'VI']

In [ ]:
#Make sure it worked!
df_data_3.State.unique()

In [ ]:
#Check out how many credentials there are.
df_data_3.Credentials.unique()

In [ ]:
#Check out the specialties.
df_data_3.Specialty.unique()

In [ ]:
#How much of the dataset is male vs female?
df_data_3.groupby('Gender').size() / df_data_3.groupby('Gender').size().sum()

In [ ]:
#How many prescribers in our dataset prescribe opioid drugs vs do not?
df_data_3.groupby('Opioid.Prescriber').size() / df_data_3.groupby('Opioid.Prescriber').size().sum()

In [ ]:
#Plot the opioid prescriber count vs non opioid prescriber count.
#The dataset has a slightly higher number of opioid prescribers.
pd.value_counts(df_data_3['Opioid.Prescriber']).plot.bar()

## Creating our Classifiers to Predict Opioid Prescribers

## Combination of Approaches: Kaggle and Indiana University

 - "Quick and Dirty" approach from Kaggle (https://www.kaggle.com/jiashenliu/quick-and-dirty-attempt-on-voting-classifier)
 - "Detecting Frequent Opioid Prescription" (https://inclass.kaggle.com/apryor6/detecting-frequent-opioid-prescription)
 - Indiana University: "Opiate prescription analysis using machine learning" (http://cgi.soic.indiana.edu/~arunsank/AML_report.pdf)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split

In [ ]:
#Find the shape of our data frame so that we know how to set our classifiers up.
print(df_data_3.shape)

In [ ]:
#Mark opioid vs non opiod drugs in df_data_3 with use of df_data_1.

In [ ]:
opioids = df_data_1 
name=opioids['Drug Name']
import re
new_name=name.apply(lambda x:re.sub("\ |-",".",str(x)))
columns=df_data_3.columns
Abandoned_variables = set(columns).intersection(set(new_name))
Kept_variable=[]
for each in columns:
    if each in Abandoned_variables:
        pass
    else:
        Kept_variable.append(each)

In [ ]:
#Look at our new shape.
df=df_data_3[Kept_variable]
print(df.shape)

In [ ]:
df.head()

In [ ]:
#Now let's remove the credentials column so that we can use the speciality column instead. 
#We will also remove the NPI column in order to trim our features down.

In [ ]:
df = df.drop(df.columns[[0, 3]], axis=1) 
df.head()

In [ ]:
#Let's now create our training and test data.
train,test = train_test_split(df,test_size=0.2,random_state=42)
print(train.shape)
print(test.shape)

In [ ]:
#Now we convert our categorical columns.
Categorical_columns=['Gender','State','Specialty']

for col in Categorical_columns:
    train[col]=pd.factorize(train[col], sort=True)[0]
    test[col] =pd.factorize(test[col],sort=True)[0]

In [ ]:
#Set our features.
features=train.iloc[:,1:242] #make sure we only use the columns that we want as our features
features.head()

In [ ]:
#Import.
import sklearn
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier

In [ ]:
#Train our models. Let's use several classifiers so that we can check out which has the highest accuracy.
#Added bagging classifier to check for overfitting (along with cross validation).
#With 'Gender' included.
features=train.iloc[:,0:242] #Make sure to remove Opioid.Prescriber (our target)!
target = train['Opioid.Prescriber']
Name=[]
Accuracy=[]
model1=LogisticRegression(random_state=22,C=0.000000001,solver='liblinear',max_iter=200)
model2=GaussianNB()
model3=RandomForestClassifier(n_estimators=200,random_state=22)
model4=GradientBoostingClassifier(n_estimators=200)
model5=KNeighborsClassifier()
model6=DecisionTreeClassifier()
model7=LinearDiscriminantAnalysis()
model8=BaggingClassifier()
Ensembled_model=VotingClassifier(estimators=[('lr', model1), ('gn', model2), ('rf', model3),('gb',model4),('kn',model5),('dt',model6),('lda',model7), ('bc',model8)], voting='hard')
for model, label in zip([model1, model2, model3, model4,model5,model6,model7,model8,Ensembled_model], ['Logistic Regression','Naive Bayes','Random Forest', 'Gradient Boosting','KNN','Decision Tree','LDA', 'Bagging Classifier', 'Ensemble']):
    scores = cross_val_score(model, features, target, cv=5, scoring='accuracy')
    Accuracy.append(scores.mean())
    Name.append(model.__class__.__name__)
    print("Accuracy: %f of model %s" % (scores.mean(),label))

In [ ]:
#Gender not included.
features=train.iloc[:,1:242] #Make sure to remove Opioid.Prescriber (our target)!
target = train['Opioid.Prescriber']
Name=[]
Accuracy=[]
model1=LogisticRegression(random_state=22,C=0.000000001,solver='liblinear',max_iter=200)
model2=GaussianNB()
model3=RandomForestClassifier(n_estimators=200,random_state=22)
model4=GradientBoostingClassifier(n_estimators=200)
model5=KNeighborsClassifier()
model6=DecisionTreeClassifier()
model7=LinearDiscriminantAnalysis()
model8=BaggingClassifier()
Ensembled_model=VotingClassifier(estimators=[('lr', model1), ('gn', model2), ('rf', model3),('gb',model4),('kn',model5),('dt',model6),('lda',model7), ('bc',model8)], voting='hard')
for model, label in zip([model1, model2, model3, model4,model5,model6,model7,model8,Ensembled_model], ['Logistic Regression','Naive Bayes','Random Forest', 'Gradient Boosting','KNN','Decision Tree','LDA', 'Bagging Classifier', 'Ensemble']):
    scores = cross_val_score(model, features, target, cv=5, scoring='accuracy')
    Accuracy.append(scores.mean())
    Name.append(model.__class__.__name__)
    print("Accuracy: %f of model %s" % (scores.mean(),label))

In [ ]:
#Looks like our highest accuracy score is 83.5% with Random Forest, followed by 83.3% with the Ensemble.

In [ ]:
#Overall, it seems our models are less accurate when 'Gender' is included, with the exception of our Ensemble which gets a fairly high accuracy at 83.4% accuracy.

In [ ]:
#Let's check out our best models from our run without 'Gender'.
Name_2=[]
Accuracy_2=[]
Ensembled_model_3=VotingClassifier(estimators=[('rf', model3),('em',Ensembled_model)], voting='hard')
for model, label in zip([model3, model4,Ensembled_model_3, model8], ['Random Forest', 'Gradient Boosting', 'Ensemble', 'Bagging Classifier']):
    scores = cross_val_score(model, features, target, cv=5, scoring='accuracy')
    Accuracy_2.append(scores.mean())
    Name_2.append(model.__class__.__name__)
    print("Accuracy: %f of model %s" % (scores.mean(),label))

In [ ]:
from sklearn.metrics import accuracy_score
classifers=[model3,model4,model8]
out_sample_accuracy=[]
Name_2=[]
for each in classifers:
    fit=each.fit(features,target)
    pred=fit.predict(test.iloc[:,1:242])
    accuracy=accuracy_score(test['Opioid.Prescriber'],pred)
    Name_2.append(each.__class__.__name__)
    out_sample_accuracy.append(accuracy)

### Evaluate

In [ ]:
#Confusion Matrix
from sklearn.metrics import confusion_matrix
y_actu = test['Opioid.Prescriber']
confusion_matrix(y_actu, pred)

In [ ]:
#Precision-Recall Curve
sklearn.metrics.precision_recall_curve(y_actu, pred, pos_label=None, sample_weight=None)

In [ ]:
#Precision-Recall Score
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_actu, pred)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

In [ ]:
#Precision-Recall Curve Plot
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

precision, recall, _ = precision_recall_curve(y_actu, pred)

plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))


After running various classifiers, we find that Random Forest, Gradient Boosting and our Ensemble models had the best performance comparatively. This means that if we were to build a larger project, we could focus on these particular classifiers, building upon them to help predict opioid prescribers (given more years of data). 

The precision-recall curve can help us determine if we were successful enough. For the unfamiliar precision-recall scores represent a balance between high recall and high precision relating to a low false positive rate and a low false negative rate respectively. When evaluating, you have four outcomes: true positive, true negative, false positive and false negative. Depending on the project, you would aim for different balances, but ideally you want everything to be as accurate, or true, as possible. In the graph above, the y axis is the precision and the x axis is recall. If the graph went straight across the middle, that would be a random-like output. Below it would be poor performance and above it would be a more accurate and better quality performance. If it were 100 it would be a perfect classifier. Because the classifier above is at 0.84 we can feel confident that our precision was good. Now I challenge you to go improve it!